In [5]:
# import necessary libraries
import pandas as pd 
import numpy as np
import nltk
import warnings
warnings.filterwarnings('ignore')
from nltk.corpus import stopwords
#from textblob import TextBlob,Word
from nltk.stem import PorterStemmer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
import pickle

Dataset Link: https://www.kaggle.com/yufengdev/bbc-fulltext-and-category

In [6]:
# read the csv file 
df=pd.read_csv(r'C:\Users\MiMs\Desktop\fileorg-master\dataset\bbc-text.csv')
df

,category,text
0,tech,tv future in the hands of viewers with home th...
1,business,worldcom boss left books alone former worldc...
2,sport,tigers wary of farrell gamble leicester say ...
3,sport,yeading face newcastle in fa cup premiership s...
4,entertainment,ocean s twelve raids box office ocean s twelve...
...,...,...
2220,business,cars pull down us retail figures us retail sal...
2221,politics,kilroy unveils immigration policy ex-chatshow ...
2222,entertainment,rem announce new glasgow concert us band rem h...
2223,politics,how political squabbles snowball it s become c...


In [7]:
# to check null values 
df.isna().any()

category    False
text        False
dtype: bool

In [8]:
# label the string values at the target values
le=LabelEncoder()
le.fit(df['category'])
df['category']=le.transform(df['category'])
df

,category,text
0,4,tv future in the hands of viewers with home th...
1,0,worldcom boss left books alone former worldc...
2,3,tigers wary of farrell gamble leicester say ...
3,3,yeading face newcastle in fa cup premiership s...
4,1,ocean s twelve raids box office ocean s twelve...
...,...,...
2220,0,cars pull down us retail figures us retail sal...
2221,2,kilroy unveils immigration policy ex-chatshow ...
2222,1,rem announce new glasgow concert us band rem h...
2223,2,how political squabbles snowball it s become c...


## label encoding 
business-0|
entertainment-1| 
politics-2|
sport-3|
tech-4

In [9]:
# count the stopwords
stop = stopwords.words('english')
def stop_word(df):
    df['stopwords_count']=df['text'].apply(lambda x : len([x for x in x.split() if x in stop]))
    print(df[['text','stopwords_count']].head())
stop_word(df)

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - 'C:\\Users\\MiMs/nltk_data'
    - 'D:\\ML\\nltk_data'
    - 'D:\\ML\\share\\nltk_data'
    - 'D:\\ML\\lib\\nltk_data'
    - 'C:\\Users\\MiMs\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [10]:
# count the no of digits 
def digits(df):
    df['count_digits']=df['text'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
    print(df[['text','count_digits']].head())
digits(df)

                                                text  count_digits
0  tv future in the hands of viewers with home th...             3
1  worldcom boss  left books alone  former worldc...             3
2  tigers wary of farrell  gamble  leicester say ...             0
3  yeading face newcastle in fa cup premiership s...             0
4  ocean s twelve raids box office ocean s twelve...             1


In [11]:
# to count the no of uppercases
def upper_case(df):
    df['count_uppercase']=df['text'].apply(lambda x: len([x for x in x.split() if x.isupper()]))
    print(df[['text','count_uppercase']].tail())
upper_case(df)
df['count_uppercase'].sum()

                                                   text  count_uppercase
2220  cars pull down us retail figures us retail sal...                0
2221  kilroy unveils immigration policy ex-chatshow ...                0
2222  rem announce new glasgow concert us band rem h...                0
2223  how political squabbles snowball it s become c...                0
2224  souness delight at euro progress boss graeme s...                0


0

In [12]:
# to remove punctuation from tweet
def punctuation_removal(df):
#     ^ : works like NOT operator
#     \w : Returns a match where the string contains any word characters 
#     \s: for whitespace
    df['text']=df['text'].str.replace('[^\w\s]','')
    print(df['text'].head())
punctuation_removal(df)

0    tv future in the hands of viewers with home th...
1    worldcom boss  left books alone  former worldc...
2    tigers wary of farrell  gamble  leicester say ...
3    yeading face newcastle in fa cup premiership s...
4    ocean s twelve raids box office ocean s twelve...
Name: text, dtype: object


In [13]:
# removal of stopwords 
def remove_stopwords(df):
    df['text']=df['text'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
    print(df['text'].head())
remove_stopwords(df)

NameError: name 'stop' is not defined

In [12]:
stop_word(df)

                                                text  stopwords_count
0  tv future hands viewers home theatre systems p...                0
1  worldcom boss left books alone former worldcom...                0
2  tigers wary farrell gamble leicester say rushe...                0
3  yeading face newcastle fa cup premiership side...                0
4  ocean twelve raids box office ocean twelve cri...                0


In [14]:
# to check the frequent words 
frequent = pd.Series(' '.join(df['text']).split()).value_counts()[:10]
frequent

the     52603
to      25002
of      19967
and     18573
a       18255
in      17629
s        9036
for      8918
is       8554
that     8255
dtype: int64

In [15]:
# removal of frequent words 
frequent=list(frequent.index)
def remove_frequent_words(df):
    df['text']=df['text'].apply(lambda x: " ".join(x for x in x.split() if x not in frequent))
    print(df['text'].head())
remove_frequent_words(df)

0    tv future hands viewers with home theatre syst...
1    worldcom boss left books alone former worldcom...
2    tigers wary farrell gamble leicester say they ...
3    yeading face newcastle fa cup premiership side...
4    ocean twelve raids box office ocean twelve cri...
Name: text, dtype: object


In [16]:
# steamming
st= PorterStemmer()
def steamming(df):
    return df['text'][0:5].apply(lambda x: " ".join(st.stem(word) for word in x.split()))
steamming(df)

0    tv futur hand viewer with home theatr system p...
1    worldcom boss left book alon former worldcom b...
2    tiger wari farrel gambl leicest say they will ...
3    yead face newcastl fa cup premiership side new...
4    ocean twelv raid box offic ocean twelv crime c...
Name: text, dtype: object

In [17]:
# lemmatization
def lemmatization(df):
    df['text']=df['text'].apply(lambda x: " ".join([Word(word).lemmatize() for word in x.split()]))
    print(df['text'].head())
lemmatization(df)

NameError: name 'Word' is not defined

In [18]:
# splitting the dataset
x=df['text']
y=df['category']
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3, random_state=0)

In [19]:
x_test

384     drive save festive holidays efforts are being ...
1983    officials respond court row australian tennis ...
985     cup holders man utd visit everton holders manc...
1386    adriano chelsea link rejected adriano agent gi...
1294    o driscollgregan lead aid stars ireland brian ...
                              ...                        
2198    asian banks halt dollar slide dollar regained ...
1256    indy buys into india paper irish publishing gr...
1637    china aviation seeks rescue deal scandalhit je...
2190    newry fight cup exit courts newry city are exp...
1581    howard truanted play snooker conservative lead...
Name: text, Length: 668, dtype: object

In [20]:
classi = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', MultinomialNB()),
])
classi.fit(x_train,y_train)
print(classi.predict(['kilroy unveils immigration policy ex-chatshow']))

[2]


In [21]:
# Save the Modle to file in the current working directory
with open('classification_model','wb') as file:
    pickle.dump(classi,file)

In [22]:
with open('classification_model','rb') as file:
    pickled_model=pickle.load(file)
pickled_model

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [25]:
pickled_model.predict(['tv future in the hands of viewers with home ','barcelona has scored a goal ','worldcom boss left books alone former'])

array([4, 3, 0])